In [23]:
import logging
import os
import pandas as pd
import re
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.linkextractors.lxmlhtml import LxmlLinkExtractor
from googlesearch import search
from datetime import time

logging.getLogger('scrapy').propagate = False

In [24]:
#import burlingame_accounts.csv and covert to list
burlingame_accounts = pd.read_csv('burlingame_accounts.csv')
burlingame_accounts = burlingame_accounts['searchterm'].tolist()

In [25]:
def get_urls(tag, n, language):
    urls = [url for url in search(tag, num_results=n, lang=language)][:n]
    return urls

In [26]:
# test function for get_urls
# get_urls('movie rating', 5 , 'en')

In [27]:
class MailSpider(scrapy.Spider):
    
    name = 'email'
    
    def parse(self, response):
        
        links = LxmlLinkExtractor(allow=()).extract_links(response)
        links = [str(link.url) for link in links]
        links.append(str(response.url))
        
        for link in links:
            yield scrapy.Request(url=link, callback=self.parse_link) 
            
    def parse_link(self, response):
        
        for word in self.reject:
            if word in str(response.url):
                return
            
        html_text = str(response.text)
        #mail_list = re.findall('\w+@\w+\.{1}\w+', html_text)
        #mail_list = re.findall('\w{3,}@\w{3,}', html_text)
        mail_list = re.findall('[a-zA-Z0-9_.+-]{3,10}@[a-zA-Z0-9-]{3,}\.[a-zA-Z0-9-.]{2,}', html_text)
        

        dic = {'email': mail_list, 'link': str(response.url)}
        df = pd.DataFrame(dic)
        
        df.to_csv(self.path, mode='a', header=False)
        df.to_csv(self.path, mode='a', header=False)

In [28]:
def ask_user(question):
    response = input(question + ' y/n' + '\n')
    if response == 'y':
        return True
    else:
        return False
def create_file(path):
    response = False
    if os.path.exists(path):
        response = ask_user('File already exists, replace?')
        if response == False: return 
    
    with open(path, 'wb') as file: 
        file.close()

In [29]:
def get_info(tag, n, language, path, reject=[]):
    
    create_file(path)
    df = pd.DataFrame(columns=['email', 'link'], index=[0])
    df.to_csv(path, mode='w', header=True)
    
    print('Collecting Google urls...')
    google_urls = get_urls(tag, n, language)
    
    print('Searching for emails...')
    process = CrawlerProcess({'USER_AGENT': 'Mozilla/5.0'})
    process.crawl(MailSpider, start_urls=google_urls, path=path, reject=reject)
    process.start()
    
    print('Cleaning emails...')
    df = pd.read_csv(path, index_col=0)
    df.columns = ['email', 'link']
    df = df.drop_duplicates(subset='email')
    df = df.reset_index(drop=True)
    df.to_csv(path, mode='w', header=True)
    
    return df

In [30]:
bad_words = ['facebook', 'instagram', 'youtube', 'twitter', 'wiki', 'doordash', 'sentry', 'toasttab']
#df = get_info('mastering studio london', 5, 'pt', 'studios.csv', reject=bad_words)

#call get_info for each account in burlingame_accounts, and delay 5 seconds between each call
for account in burlingame_accounts:
    get_info(account, 2, 'pt', 'burlingame_emails.csv', reject=bad_words)
    time.sleep(5)

2022-11-23 13:29:32 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.google.com:443
2022-11-23 13:29:33 [urllib3.connectionpool] DEBUG: https://www.google.com:443 "GET /search?q=226%2BKITCHEN&num=4&hl=pt&start=0 HTTP/1.1" 200 None


2022-11-23 13:29:33 [py.warnings] WARNING: C:\Users\wheel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scrapy\utils\request.py:231: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)



Searching for emails...


ReactorNotRestartable: 

In [ ]:
#df.head()